# Misra–Gries (Algorithm 3.1)

目的: ストリームから**頻出要素（heavy hitters）**を近似検出。メモリは `k-1` 個のカウンタ。

処理:
1. アイテムが既存のキーならカウントを+1  
2. 空きがあるなら新規キーを追加（カウント=1）  
3. 空きがないなら**全カウンタを1ずつ減算**し、0になったキーを削除  

性質: 出力された候補の真の頻度 `f(x)` は概ね `⌊ n/k ⌋` の誤差で下界化できる。


In [ ]:
from collections import defaultdict, Counter

def misra_gries(stream, k):
    # maintain at most k-1 counters
    counters = {}
    n = 0
    for x in stream:
        n += 1
        if x in counters:
            counters[x] += 1
        elif len(counters) < k-1:
            counters[x] = 1
        else:
            # decrement all
            to_del = []
            for j in list(counters.keys()):
                counters[j] -= 1
                if counters[j] == 0:
                    to_del.append(j)
            for j in to_del:
                del counters[j]
    return counters, n

# demo on chat authors
import pandas as pd
# Corrected file path to use an existing sample file
df = pd.read_csv("/content/sample_data/california_housing_train.csv", encoding="utf-8")
stream = df["longitude"].astype(str).tolist() # Using 'longitude' column for demonstration
k = 20
counters, n = misra_gries(stream, k)

# compare with exact counts
true_counts = Counter(stream)
candidates = {u: true_counts[u] for u in counters.keys()}
top_true = true_counts.most_common(20)

print("n =", n, "unique =", len(true_counts))
print("MG candidates (|T|={}):".format(len(counters)))
print(sorted([(u, counters[u], candidates[u]) for u in counters], key=lambda t: -candidates[t[0]])[:20])
print("\nTop-20 ground truth:")
print(top_true)

n = 17000 unique = 827
MG candidates (|T|=15):
[('-124.16', 9, 11), ('-124.14', 8, 10), ('-124.17', 8, 10), ('-124.15', 6, 8), ('-124.09', 4, 7), ('-124.1', 4, 7), ('-124.08', 3, 6), ('-124.13', 4, 6), ('-124.11', 2, 5), ('-124.18', 3, 5), ('-124.19', 2, 4), ('-124.21', 2, 3), ('-124.23', 2, 3), ('-124.3', 2, 2), ('-124.35', 1, 1)]

Top-20 ground truth:
[('-118.31', 136), ('-118.3', 128), ('-118.32', 124), ('-118.29', 118), ('-118.35', 116), ('-118.36', 115), ('-118.27', 114), ('-118.28', 113), ('-118.37', 111), ('-118.19', 110), ('-118.34', 107), ('-118.2', 105), ('-118.25', 102), ('-118.13', 101), ('-118.18', 101), ('-118.14', 98), ('-118.15', 98), ('-118.12', 94), ('-118.21', 91), ('-118.26', 90)]


### コードの説明

このコードは、ストリームデータの中から頻繁に出現する要素（heavy hitters）を近似的に見つけ出す「Misra–Gries アルゴリズム」を実装。

1.  **`misra_gries(stream, k)` 関数**:
    *   入力: データのリスト (`stream`) と、保持するカウンターの最大数 (`k`)。
    *   処理: ストリームを順番に処理し、要素の出現回数を数える。カウンターの数が `k-1` を超えた場合は、すべてのカウンターの値を1減らし、値が0になったカウンターを削除。これにより、メモリ使用量を抑えつつ、頻出要素を近似的に捉える。
    *   出力: 近似的な出現回数を持つ要素の辞書と、ストリームの全体の要素数 (`n`)。

2.  **デモの実行**:
    *   サンプルデータ (`california_housing_train.csv`) を読み込み、`longitude` 列をストリームデータとして使用。
    *   `misra_gries` 関数を使って、頻出する `longitude` の値を近似的に見つける（ここでは `k=20`）。
    *   正確な出現回数を `Counter` で計算し、Misra-Gries アルゴリズムの結果と比較。

3.  **結果の出力**:
    *   ストリームの要素数 (`n`) とユニークな要素数 (`unique`) を表示。
    *   Misra-Gries アルゴリズムが見つけた候補 (`MG candidates`) と、それぞれの近似カウント、正確なカウントを表示。
    *   正確な出現回数の上位20位 (`Top-20 ground truth`) を表示し、Misra-Gries の結果と比較できるようにしている。

このアルゴリズムは、非常に大きなストリームデータに対して、メモリを節約しながら頻出要素を見つけたい場合に役立つ。